In [1]:
import pandas as pd
import numpy as np
import math

import warnings
warnings.filterwarnings('ignore')

In [2]:
articles_df = pd.read_csv('./data/shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']

In [3]:
interactions_df = pd.read_csv('./data/users_interactions.csv')

In [4]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [5]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [6]:
interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

In [7]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

In [8]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [9]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].max()
)
        
interactions_full_df = interactions_full_df.reset_index()

In [10]:
from sklearn.model_selection import train_test_split

split_ts = 1475519545
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()


In [11]:
# Задание 6.1
# Найдите оценку взаимодействия пользователя с ID - 1032019229384696495 со статьёй с ID 943818026930898372 . Результат округлите до
# двух знаков после точки-разделителя.

# Примечание. Здесь и далее (пока не будет указано иное) необходимо работать с обучающей выборкой.

ratings = pd.pivot_table(
    interactions_train_df,
    values="eventStrength",
    index="personId",
    columns="contentId",
).fillna(0)
round(ratings.loc["-1032019229384696495", "943818026930898372"], 2)

2.32

In [12]:
# Теперь давайте попробуем применить memory-based-подход коллаборативной фильтрации.
# Примечание. Данных достаточно много, поэтому для увеличения скорости работы преобразуйте таблицу в массив numpy.

In [13]:
# Задание 6.2
# Найдите среднее арифметическое всех чисел в получившемся массиве. Результат округлите до трёх знаков после точки-разделителя.
ratings_m = ratings.values
round(ratings_m.mean(), 3)

0.017

Перейдём к реализации коллаборативной фильтрации. Ранее мы делали это с помощью библиотеки surprise, однако это не всегда
удобно, так как эта библиотека имеет ограниченное количество метрик для оценки качества и небольшой потенциал для более тонкой
настройки алгоритма. Поэтому давайте попробуем реализовать алгоритмы коллаборативной фильтрации « нуля». Такая практика
применяется, если необходимо выстроить более сложную систему, чем могут предложить готовые модули. Кроме того, «ручная»
реализация алгоритмов позволит лучше понять принцип их работы

In [14]:
ratings

contentId,-1006791494035379303,-1021685224930603833,-1022885988494278200,-1024046541613287684,-1033806831489252007,-1038011342017850,-1039912738963181810,-1046621686880462790,-1051830303851697653,-1055630159212837930,...,9217155070834564627,921770761777842242,9220445660318725468,9222265156747237864,943818026930898372,957332268361319692,966067567430037498,972258375127367383,980458131533897249,98528655405030624
personId,,,,,,,,,,,,,,,,,,,,,
-1007001694607905623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
-1032019229384696495,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,2.321928,0.0,0.0,0.0,0.0,0.0
-108842214936804958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,2.0,0.0,0.0,0.0
-1130272294246983140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0
-1160159014793528221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953707509720613429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
983095443598229476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
989049974880576288,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [15]:
# Задание 6.3
# Постройте матрицу схожести. Для этого вычислите все попарные коэффициенты корреляции для матрицы, полученной в предыдущем задании.
# Для каждой пары учитывайте только ненулевые значения (так как нулевые обозначают отсутствие взаимодействия и не интересуют нас).
# Выведите результат, полученный в ячейке с третьим индексом по строкам и сороковым - по столбцам.

similarity_users = np.zeros((len(ratings_m), len(ratings_m)))
for i in (range(len(ratings_m)-1)):
    for j in range(i+1, len(ratings_m)):
     
        mask_uv = (ratings_m[i] != 0) & (ratings_m[j] != 0)
        ratings_v = ratings_m[i, mask_uv]
        ratings_u = ratings_m[j, mask_uv]

        similarity_users[i,j] = np.corrcoef(ratings_v, ratings_u)[0, 1]
        similarity_users[j,i] = similarity_users[i,j]

similarity_users[3,40]

-0.3333333333333333

In [16]:
# Задание 6.4
# Для каждого пользователя:
# 1. Найти пользователей с похожестью больше 0.
# 2. Для каждой статьи вычислить долю пользователей (среди выделенных на первом шаге), которые взаимодействовали со статьёй.
# 3. Порекомендовать статьи (не более 10) с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел).
# •Постройте рекомендательную систему по алгоритму, описанному выше. Найдите первую рекомендацию для строки 35 (если считать с нуля).

interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)
 
interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)
 
interactions['true_test'] = [ [] if x is np.NaN else x for x in interactions['true_test'] ]

prediction_user_based = []
for i in range(len(similarity_users)):
    users_sim = similarity_users[i] > 0
    if not any(users_sim):
        prediction_user_based.append([])
    else:
        tmp_recommend = np.argsort(ratings_m[users_sim].sum(axis=0))[::-1]
        tmp_recommend = ratings.columns[tmp_recommend]
        recommend = np.array(tmp_recommend)[~np.in1d(tmp_recommend, interactions.iloc[i]["true_train"])][:10]
        prediction_user_based.append(list(recommend))
interactions['prediction_user_based'] = prediction_user_based
prediction_user_based[35][0]

'-5148591903395022444'

In [17]:
# После того как сделаны предсказания, можно вычислить качество по метрике, которую мы определили в предыдущем модуле при решении этой задачи:

def calc_precision(column):
    return ( interactions.apply(  lambda row:len(set(row['true_test']).intersection(
                set(row[column]))) /min(len(row['true_test']) + 0.001, 10.0), axis=1)).mean()

In [18]:
# Задание 6.5
# Вычислите точность полученного предсказания. Ответ округлите до трёх знаков после точки-разделителя.
round(calc_precision('prediction_user_based'), 3)

0.005

In [19]:
# Задание 6.6
# Теперь реализуем рекомендательную систему с использованием SVD.
# Разложите матрицу взаимодействий пользователей со статьями с помощью функции svd из модуля scip. Найдите максимальное значение в
# получившейся матрице U. Результат округлите до двух знаков после точки-разделителя.
from scipy.linalg import svd
U, sigma, V = svd(ratings)
U.max()

0.7071067811865471

Значения матрицы с сингулярными числами отсортированы по убыванию. Допустим, мы хотим оставить только первые 100 компонент и
получить скрытые представления размерности 100. Для этого необходимо оставить 100 столбцов в матрице U, только первые 100
значений из sigma (и сделать из них диагональную матрицу) и 100 строк в матрице V. Затем необходимо перемножить преобразованные
матрицы.

In [20]:
# Задание 6.7
# Найдите сумму всех элементов в новой сингулярной матрице. Ответ округлите до двух знаков после точки-разделителя.
k = 100
s = np.diag(sigma[:k])
U = U[:, 0:k]
V = V[0:k, :]

round(s.sum(), 2)

2096.43

Теперь мы можем сделать предсказание по полученной матрице.

Примечание. Помните, что не нужно учитывать статьи, которые уже были просмотрены пользователем.

Найдите для каждого пользователя статьи с наибольшими оценками в восстановленной матрице.

In [21]:
# Задание 6.8
# Вычислите качество полученного предсказания, используя всё ту же метрику точности. Ответ округлите до трёх знаков после точки-
# разделителя.

new_ratings = pd.DataFrame(
    U.dot(s).dot(V), index=ratings.index, columns=ratings.columns
)
top_k = 10
predictions = []

for personId in interactions.index:
    prediction = (
        new_ratings.loc[personId].sort_values(ascending=False).index.values
    )

    predictions.append(
        list(
            prediction[
                ~np.in1d(prediction, interactions.loc[personId, "true_train"])
            ]
        )[:top_k]
    )

interactions["prediction_svd"] = predictions

calc_precision("prediction_svd")

0.012023152289763838

Итак, мы реализовали два алгоритма коллаборативной фильтрации буквально с нуля! Теперь для полноты картины давайте реализуем
на этих данных гибридную модель и посмотрим, какое качество получится. Для этого воспользуемся уже изученной библиотекой LightFM.

In [22]:
# Задание 6.9
# Возьмите матрицу, подготовленную в задании 6.1. Преобразуйте её в разреженную матрицу:
from scipy.sparse import csr_matrix
ratings_matrix = csr_matrix(ratings)

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k  
from scipy.sparse import csr_matrix

ratings_matrix = csr_matrix(ratings) # передаём в качестве аргументов в функцию выставленный рейтинг (это будут значения матрицы), а также id пользователя и id книги (это будут индексы для строк и столбцов матрицы)
model = LightFM(loss='warp', #определяем функцию потерь
                random_state=13, #фиксируем случайное разбиение
                learning_rate=0.05, #темп обучения
                no_components=100) #размерность вектора для представления данных в модели

train,test = random_train_test_split(ratings_matrix, test_percentage=0.3, random_state=13)
model.fit(train)
prec_score = precision_at_k(model, test).mean()


print(round(prec_score, 2))

0.04
